In [ ]:
import numpy as np
from keras.layers import Input, Convolution2D, Dense, Dropout, Flatten, concatenate, BatchNormalization
from keras.models import Model  # basic class for specifying and training a neural network
from keras import losses
import keras

import tensorflow as tf

In [ ]:
SIZE = 13

In [5]:
npz = np.load('models/train_vectors_13_13_r2.npz')
train_vectors = npz['train_vectors']
train_p = npz['train_p']
train_q = npz['train_q']

In [6]:
inp = Input(shape=(SIZE, SIZE, 1))

bn1 = BatchNormalization()(inp)
conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn1)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn3)
bn4 = BatchNormalization()(conv_3)

flat = Flatten()(bn4)
turn_input = Input(shape=(1,), name='turn')
full = concatenate([flat, turn_input])

hidden = Dense(15, activation='relu', kernel_initializer='random_uniform')(full)
bn4 = BatchNormalization()(hidden)

out = Dense(1)(bn4)

q_model = Model(inputs=[inp, turn_input], outputs=out)
q_model.compile(loss=losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])

In [49]:
boards = np.array([board for board in train_vectors[:, 0]]).reshape(train_vectors.shape[0], SIZE, SIZE, 1)


In [51]:
with tf.device('/gpu:0'):
    model.fit([boards, 
               train_vectors[:, 1]], 
              train_q, 
              epochs=10,
             validation_split=0.1)

Train on 52358 samples, validate on 5818 samples
Epoch 1/10
52358/52358 [==============================] - 14s 259us/step - loss: 0.0181 - mean_squared_error: 0.0181 - val_loss: 0.0160 - val_mean_squared_error: 0.0160
Epoch 2/10
52358/52358 [==============================] - 13s 257us/step - loss: 0.0151 - mean_squared_error: 0.0151 - val_loss: 0.0114 - val_mean_squared_error: 0.0114
Epoch 3/10
52358/52358 [==============================] - 13s 256us/step - loss: 0.0133 - mean_squared_error: 0.0133 - val_loss: 0.0145 - val_mean_squared_error: 0.0145
Epoch 4/10
52358/52358 [==============================] - 13s 253us/step - loss: 0.0121 - mean_squared_error: 0.0121 - val_loss: 0.0094 - val_mean_squared_error: 0.0094
Epoch 5/10
52358/52358 [==============================] - 13s 255us/step - loss: 0.0106 - mean_squared_error: 0.0106 - val_loss: 0.0095 - val_mean_squared_error: 0.0095
Epoch 6/10
52358/52358 [==============================] - 13s 257us/step - loss: 0.0096 - mean_squared_err

In [57]:
model.save('models/13_13_q_v2.net')

In [54]:
inp = Input(shape=(SIZE, SIZE, 1))

bn1 = BatchNormalization()(inp)
conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn1)
bn2 = BatchNormalization()(conv_1)
conv_2 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn2)
bn3 = BatchNormalization()(conv_2)
conv_3 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                       kernel_initializer='random_uniform')(bn3)
bn4 = BatchNormalization()(conv_3)

flat = Flatten()(bn4)
turn_input = Input(shape=(1,), name='turn')
full = concatenate([flat, turn_input])

hidden = Dense(15, activation='relu', kernel_initializer='random_uniform')(full)
bn4 = BatchNormalization()(hidden)

out = Dense(SIZE ** 2, activation='softmax')(bn4)

p_model = Model(inputs=[inp, turn_input], outputs=out)
p_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [55]:
with tf.device('/gpu:0'):
    p_model.fit([boards, 
               train_vectors[:, 1]], 
              train_p, 
              epochs=10,
             validation_split=0.1)

Train on 52358 samples, validate on 5818 samples
Epoch 1/10
52358/52358 [==============================] - 15s 296us/step - loss: 3.9676 - acc: 0.1580 - val_loss: 3.2680 - val_acc: 0.2711
Epoch 2/10
52358/52358 [==============================] - 14s 269us/step - loss: 3.2059 - acc: 0.2865 - val_loss: 3.0075 - val_acc: 0.3118
Epoch 3/10
52358/52358 [==============================] - 14s 264us/step - loss: 2.9485 - acc: 0.3353 - val_loss: 2.8870 - val_acc: 0.3501
Epoch 4/10
52358/52358 [==============================] - 14s 260us/step - loss: 2.7680 - acc: 0.3700 - val_loss: 2.8445 - val_acc: 0.3781
Epoch 5/10
52358/52358 [==============================] - 14s 259us/step - loss: 2.6233 - acc: 0.3947 - val_loss: 2.8634 - val_acc: 0.3737
Epoch 6/10
52358/52358 [==============================] - 13s 254us/step - loss: 2.5032 - acc: 0.4145 - val_loss: 2.9012 - val_acc: 0.3805
Epoch 7/10
52358/52358 [==============================] - 13s 257us/step - loss: 2.4022 - acc: 0.4318 - val_loss: 2.8

In [58]:
p_model.save('models/13_13_p_v2.net')